In [1]:
base_url = 'https://api.pushshift.io/reddit/submission/search'

In [82]:
import requests
import pandas as pd
from datetime import datetime
import time

In [83]:
params = {
    'subreddit' : 'superhoops',
    'limit' : 1000,
    'unitl' : round(time.time()),
    'sort' : 'created_utc',
    'order' : 'desc'
    
}

In [84]:
res = requests.get(base_url, params)

In [67]:
res.json()['metadata']

{'es': {'took': 1139,
  'timed_out': False,
  '_shards': {'total': 352, 'successful': 352, 'skipped': 0, 'failed': 0},
  'hits': {'total': {'value': 7592, 'relation': 'eq'}, 'max_score': None}},
 'es_query': {'size': 100,
  'query': {'bool': {'must': [{'bool': {'must': [{'range': {'created_utc': {'gte': 0}}}]}},
     {'bool': {'should': [{'match': {'subreddit': 'superhoops'}}],
       'minimum_should_match': 1}}]}},
  'aggs': {},
  'sort': {'created_utc': 'asc'}},
 'es_query2': '{"size":100,"query":{"bool":{"must":[{"bool":{"must":[{"range":{"created_utc":{"gte":0}}}]}},{"bool":{"should":[{"match":{"subreddit":"superhoops"}}],"minimum_should_match":1}}]}},"aggs":{},"sort":{"created_utc":"asc"}}'}

In [85]:
df = pd.DataFrame(res.json()['data'])

In [69]:
df.columns

Index(['downs', 'link_flair_text', 'distinguished', 'media', 'url',
       'link_flair_css_class', 'id', 'edited', 'num_reports', 'created_utc',
       'subreddit', 'title', 'author_flair_text', 'is_self', 'author',
       'media_embed', 'permalink', 'author_flair_css_class', 'selftext',
       'domain', 'num_comments', 'likes', 'thumbnail', 'saved', 'subreddit_id',
       'ups', 'approved_by', 'score', 'selftext_html', 'created', 'hidden',
       'over_18', 'retrieved_utc', 'updated_utc', 'utc_datetime_str',
       'edited_on'],
      dtype='object')

In [86]:
df.selftext.value_counts()

In [88]:
datetime.fromtimestamp(df['created_utc'].min())

datetime.datetime(2018, 7, 1, 12, 1, 22)

In [105]:
%time

all_posts = []
current_time = round(time.time())
params_until = {
    'subreddit': 'superhoops',
    'limit': 1000,
    'until': current_time,
    'sort': 'created_utc',
    'order': 'desc'
}

num_posts = 0
total_time = 0

for _ in range(8):
    start = time.time()
    res = requests.get(base_url, params_until)
    print(res.status_code)
    try:
        posts = pd.DataFrame(res.json()['data'])
        all_posts.append(posts)
        params['until'] = posts['created_utc'].min()
        num_posts += len(posts)
        end = time.time()
        time_to = end - start
        total_time += time_to
    except:
        pass
    print(f"time to scrape: {time_to}; num posts: {num_posts}")

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs
200
time to scrape: 3.552037000656128; num posts: 1000
200
time to scrape: 3.6317203044891357; num posts: 2000
200
time to scrape: 2.766116142272949; num posts: 3000
200
time to scrape: 2.606278896331787; num posts: 4000
200
time to scrape: 3.19620680809021; num posts: 5000
200
time to scrape: 2.7783219814300537; num posts: 6000
200
time to scrape: 2.715489149093628; num posts: 7000
200
time to scrape: 0.5317811965942383; num posts: 8000


In [106]:
total_time/ 8

2.7222439348697662